In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import json


In [2]:
def get_12Moforecast(tick):
    """
    Takes in a tick sign and date variable and gets the historical dividend dates for that tick symbal AFTER the data input.
    Outputs: dataframe with these columns:
    'exOrEffDate', 'type', 'amount', 'declarationDate', 'recordDate','paymentDate', 'Tick'
    """
    
    #need headers otherwise they will think you are a bot 
    headers = { "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "Upgrade-Insecure-Requests": "1",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 RuxitSynthetic/1.0 v7992529445 t38550 ath9b965f92 altpub cvcv=2"
        }

    url = "https://money.cnn.com/quote/forecast/forecast.html?symb={}".format(tick)
    response = requests.get(url,headers=headers)
    
    return response

In [3]:
def parse_12Moforecast(tick, response):

    html_text = response.text
    
    columnNames = ['tick', 'Analysts', 'Recommendation', 'Median', 'High', 'Low', 'Growth', 'Last']
    
    soup = BeautifulSoup(html_text, "html.parser")
    
    # find all div elements that are inside a div element
    # and are proceeded by an h3 element
    selector = 'div > h3 ~ div'
    
    divs = soup.select(selector)
    
    # Extract data from the found elements
    data = [x.text.split(';')[-1].strip() for x in divs]
    
    df = pd.DataFrame(columns = columnNames)

    for x in data:
        
        if (x.find("12-month price forecasts for") > 0):
            matches = re.findall(r'[\d\.\d]+', x)
            df.loc[1, 'Median'] = matches[2]
            df.loc[1, 'High'] = matches[3]
            df.loc[1, 'Low'] = matches[4]
            df.loc[1, 'Growth'] = matches[5]
            df.loc[1, 'Last'] = matches[6]
            
        if (x.startswith("The current consensus among") == 1):
            match = re.findall(r'[\d\.\d]+', x)    
            df.loc[1, 'Analysts'] = match[1]
            if (x.find('is to Hold stock') > 0):
                df.loc[1, 'Recommendation'] = 'Hold'
            if (x.find('is to Buy stock') > 0):
                df.loc[1, 'Recommendation'] = 'Buy'
            if (x.find('is to Sell stock') > 0):
                df.loc[1, 'Recommendation'] = 'Sell'


    
    df.loc[1, 'tick'] = tick
           
    return df


In [20]:
def get_tickers():

    #need headers otherwise they will think you are a bot 
    headers = { "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "Upgrade-Insecure-Requests": "1",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 RuxitSynthetic/1.0 v7992529445 t38550 ath9b965f92 altpub cvcv=2"
        }

    url = "https://www.thestreet.com/util/ratings-screener.jsp?wt=json&rows=200&indent=off&q=type:equity+AND+LetterGradeSort:[+90+TO+100+]+AND+Growth:[+0+TO+5+]++AND+TotalReturn:[+0+TO+5+]++AND+Efficiency:[+0+TO+5+]++AND+Pricevolatility:[+0+TO+5+]++AND+Solvency:[+0+TO+5+]++AND+Income:[+0+TO+5+]+&fl=ticker,issue_name,LetterGradeRating,CurrentRating,LetterGradeSort,type,Risk,recommendation,exchange&sort=ticker+asc"
    response = requests.get(url,headers=headers)

    json_data = json.loads(response.text)

    json_divs = json_data['response']['docs']
    df = pd.json_normalize(json_divs)
        
    return df

In [ ]:
dfs = []
ser = []
ser = get_tickers()
for stock in ser['ticker']:
    response = get_12Moforecast(stock)
    df = parse_12Moforecast(stock, response)
    dfs.append(df)
    
    

       type ticker                    issue_name exchange CurrentRating  \
0    equity      A      AGILENT TECHNOLOGIES INC     NYSE           Buy   
1    equity   ABBV                    ABBVIE INC     NYSE           Buy   
2    equity    ABT           ABBOTT LABORATORIES     NYSE           Buy   
3    equity    ACN                 ACCENTURE PLC     NYSE           Buy   
4    equity   ADBE                     ADOBE INC   NASDAQ           Buy   
..      ...    ...                           ...      ...           ...   
153  equity  WSO.B                    WATSCO INC     NYSE           Buy   
154  equity    WTS  WATTS WATER TECHNOLOGIES INC     NYSE           Buy   
155  equity    XEL               XCEL ENERGY INC   NASDAQ           Buy   
156  equity   YORW                 YORK WATER CO   NASDAQ           Buy   
157  equity   ZBRA         ZEBRA TECHNOLOGIES CP   NASDAQ           Buy   

     LetterGradeSort LetterGradeRating  
0                 90                A-  
1                

In [ ]:
result = pd.concat(dfs, ignore_index=True)
result.to_csv('12MoForecast.csv',index=False)
result